# MNIST Dataset

In [2]:
import torch   #general pytorch
import torch.nn as nn  #neural network module
import torch.nn.functional as F  #useful functions like softmax, or relu

#pip install torchvision; conda install torchvision
from torchvision import datasets, transforms  #transforms for image processing
from torch.utils.data import DataLoader       #dataloader for preparing batch

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


## 1. ETL

In [3]:
#load the dataset
#DON'T FOLLOW MY ../data, put where you want.....

#train basically mean is this training set?
#download is download....
#transform means do you want to apply any processing to these images

#for the transform, we gonna just convert to tensor
transform = transforms.ToTensor()  #this NOT only converts to tensor, but also 
#converts from 0 to 255 to 0 to 1

train = datasets.MNIST(root="../data", train=True, download=True, transform = transform)